## 1. Configuraciones iniciales

In [1]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

### 1.1 Leer archivo de configuración

In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

### 1.2 Acceder a base de datos transaccional

In [3]:
RDS_HOSTNAME = 'e-dreams-transactional.cfum2oeu0d6b.us-east-1.rds.amazonaws.com'

In [4]:
driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOSTNAME}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

## 2. Crear las dimensiones

### 2.1 Dimensión de artículos

#### 2.1.1 Tabla articulo

In [5]:
query = 'SELECT * FROM articulo;'
df_articulos = pd.read_sql(query, driver)
df_articulos

,id_articulo,id_categoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,0,10010,66883,among,58.97,61,Hotel off security design second. Born at cour...,Agree himself.,False
1,1,10015,86212,run,94.42,38,Entire air final because relationship. Minute ...,Check sense support.,True
2,2,10015,84042,agreement,93.58,48,Material appear there recognize. Level perhaps...,Enough avoid with.,False
3,3,10025,61637,treat,74.02,61,Tax senior half marriage well. Thousand decide...,Positive.,True
4,4,10025,40882,image,41.02,71,Report strategy soldier prevent will executive...,Develop mention.,False
...,...,...,...,...,...,...,...,...,...
995,995,10018,36947,career,37.44,90,Ask garden method last than nothing suddenly a...,Protect want then.,True
996,996,10008,74981,finally,45.04,27,Resource small prepare send stage church tradi...,Specific near land.,False
997,997,10025,39489,nice,72.40,46,Evidence issue field subject seem film. Few to...,Loss yard once help.,False
998,998,10025,27450,central,28.28,1,Song minute sell identify focus among media. H...,Teach relationship.,True


In [6]:
df_articulos = df_articulos.drop(columns=['stock', 'estado_articulo'])
df_articulos

,id_articulo,id_categoria,codigo,nombre,precio_venta,descripcion,imagen
0,0,10010,66883,among,58.97,Hotel off security design second. Born at cour...,Agree himself.
1,1,10015,86212,run,94.42,Entire air final because relationship. Minute ...,Check sense support.
2,2,10015,84042,agreement,93.58,Material appear there recognize. Level perhaps...,Enough avoid with.
3,3,10025,61637,treat,74.02,Tax senior half marriage well. Thousand decide...,Positive.
4,4,10025,40882,image,41.02,Report strategy soldier prevent will executive...,Develop mention.
...,...,...,...,...,...,...,...
995,995,10018,36947,career,37.44,Ask garden method last than nothing suddenly a...,Protect want then.
996,996,10008,74981,finally,45.04,Resource small prepare send stage church tradi...,Specific near land.
997,997,10025,39489,nice,72.40,Evidence issue field subject seem film. Few to...,Loss yard once help.
998,998,10025,27450,central,28.28,Song minute sell identify focus among media. H...,Teach relationship.


#### 2.1.2 Tabla categoria

In [7]:
query = 'SELECT * FROM categoria;'
df_categoria = pd.read_sql(query, driver)
df_categoria

,id_categoria,nombre,descripcion,estado_categoria
0,10008,comestibles,comida,True
1,10010,AlimentoMascotas,Mascotas,True
2,10015,Golosinas,"Chocolates, dulces",True
3,10018,MuebleCaja,articulos de muebles de caja,True
4,10025,gorros,gorros para frio,False


In [8]:
df_categoria = df_categoria.drop(columns=['estado_categoria'])
df_categoria = df_categoria.rename(columns={'nombre': 'nombre_categoria', 'descripcion': 'descripcion_categoria'})
df_categoria

,id_categoria,nombre_categoria,descripcion_categoria
0,10008,comestibles,comida
1,10010,AlimentoMascotas,Mascotas
2,10015,Golosinas,"Chocolates, dulces"
3,10018,MuebleCaja,articulos de muebles de caja
4,10025,gorros,gorros para frio


#### 2.1.2 Crear la dimension

In [9]:
dim_articulo = df_articulos.merge(df_categoria, on='id_categoria', how='inner')
dim_articulo

,id_articulo,id_categoria,codigo,nombre,precio_venta,descripcion,imagen,nombre_categoria,descripcion_categoria
0,0,10010,66883,among,58.97,Hotel off security design second. Born at cour...,Agree himself.,AlimentoMascotas,Mascotas
1,12,10010,49381,lose,53.39,Pressure report same president see young. Avoi...,Well realize fish.,AlimentoMascotas,Mascotas
2,13,10010,16616,cell,82.76,Ball great matter. Plan national send room end...,Wonder discussion.,AlimentoMascotas,Mascotas
3,14,10010,77174,son,30.26,Back throughout what fly generation day. Thus ...,Method economy man.,AlimentoMascotas,Mascotas
4,16,10010,70720,off,94.47,Street home effort again news according. Bad s...,Including here city.,AlimentoMascotas,Mascotas
...,...,...,...,...,...,...,...,...,...
995,981,10008,11666,what,59.30,South bag entire message. Beautiful anything m...,Behavior record.,comestibles,comida
996,986,10008,36033,sort,40.34,Themselves several society specific research. ...,Church seem with.,comestibles,comida
997,990,10008,12349,whole,61.85,Whole age central PM police camera compare mee...,Next could talk.,comestibles,comida
998,994,10008,15604,event,67.85,Long series plant reveal natural collection li...,At trip test hard.,comestibles,comida


### 2.2 Dimensión de personas

#### 2.2.1 Crear la dimension

In [10]:
query = 'SELECT * FROM persona;'
dim_personas = pd.read_sql(query, driver)
dim_personas

,id_persona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,0,Juridica,Collin Pope,Licencia,892-95-2112,634 Benson Circle\nGr,90976019,brooksmaria@yahoo.co
1,1,Individual,Joel Campbell,Licencia,704-56-0410,7923 Carr Well\nLake,41255764,whitakerjennifer@gma
2,2,Juridica,Daniel Lopez,DPI,033-21-0987,08455 Eric Hills Apt,66528770,urusso@hotmail.com
3,3,Juridica,Melissa Larsen,DPI,402-40-5373,008 Kennedy Roads Ap,73762003,michelle89@gmail.com
4,4,Juridica,Frank Martinez,DPI,233-12-7368,2711 Reynolds Causew,25501899,josephadams@gmail.co
5,5,Juridica,Ashley Moore,DPI,280-16-3603,126 Beltran Parkway\n,89925465,gabriellathornton@gm
6,6,Individual,James Martinez,DPI,638-35-1288,227 Sean Lane\nLake S,71380723,sharon24@gmail.com
7,7,Individual,Bryan Gibson,Licencia,130-85-6202,"PSC 2911, Box 2871\nA",73886843,christina98@gmail.co
8,8,Juridica,Lauren Wells PhD,Licencia,354-29-0070,66297 Beck Points Su,85580160,floresbrett@gmail.co
9,9,Individual,Brenda Stevenson,DPI,877-53-9451,655 David Fall\nSouth,92796495,xjohnson@yahoo.com


### 2.3 Dimensión de usuario

#### 2.3.1 Tabla usuario

In [11]:
query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(query, driver)
df_usuario

,id_usuario,id_rol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado_usuario
0,0,14586,Jason Collins,Licencia,171-37-6240,167 Kim Mills Suite,66466083,josephaustin@gmail.c,"[b'x', b'6', b'3', b'n', b'F', b'V', b'T', b'y...",False
1,1,14586,Kelly Nelson,DPI,317-88-6562,55279 Gibson Forge S,22196109,cheryl25@hotmail.com,"[b'&', b'A', b'J', b'd', b'9', b'O', b'k', b'L...",True
2,2,18568,Randy Lee,Licencia,872-50-5771,7265 Simmons Estate,41975722,ginarivers@yahoo.com,"[b'X', b'B', b'7', b'q', b'R', b'r', b')', b'K...",False
3,3,14586,Christopher Hardy,DPI,084-04-7482,05077 Thomas Square\n,27287390,amanda48@gmail.com,"[b')', b'm', b'6', b'5', b'7', b'v', b'a', b'p...",False
4,4,16578,Jason West,Licencia,631-57-6208,1654 Adams Spring Ap,81139864,matthewcarr@yahoo.co,"[b'Z', b'3', b'f', b'D', b'8', b'o', b'2', b'b...",True
...,...,...,...,...,...,...,...,...,...,...
9995,9995,16578,Ann Martin,DPI,590-34-9478,55399 Ryan Track\nDan,39241256,roachwendy@gmail.com,"[b'Z', b'v', b'I', b'R', b'g', b'T', b'1', b'X...",True
9996,9996,16578,Rickey Cruz,Licencia,683-13-5659,799 Barnes Rue\nNorth,57820290,susan55@hotmail.com,"[b'B', b'H', b'N', b'O', b'S', b'9', b'N', b'm...",True
9997,9997,14586,Hannah Sanchez,Licencia,681-85-8890,059 Patty Island\nWes,11249491,rmorgan@yahoo.com,"[b'!', b'e', b'6', b'7', b'R', b'd', b'h', b'B...",False
9998,9998,18568,Brian Orr,Licencia,846-47-3146,95472 Grimes Street\n,68817726,patriciabrown@yahoo.,"[b'J', b'O', b'%', b'x', b'(', b'D', b'b', b'd...",False


In [12]:
df_usuario = df_usuario.drop(columns=['estado_usuario'])
df_usuario

,id_usuario,id_rol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave
0,0,14586,Jason Collins,Licencia,171-37-6240,167 Kim Mills Suite,66466083,josephaustin@gmail.c,"[b'x', b'6', b'3', b'n', b'F', b'V', b'T', b'y..."
1,1,14586,Kelly Nelson,DPI,317-88-6562,55279 Gibson Forge S,22196109,cheryl25@hotmail.com,"[b'&', b'A', b'J', b'd', b'9', b'O', b'k', b'L..."
2,2,18568,Randy Lee,Licencia,872-50-5771,7265 Simmons Estate,41975722,ginarivers@yahoo.com,"[b'X', b'B', b'7', b'q', b'R', b'r', b')', b'K..."
3,3,14586,Christopher Hardy,DPI,084-04-7482,05077 Thomas Square\n,27287390,amanda48@gmail.com,"[b')', b'm', b'6', b'5', b'7', b'v', b'a', b'p..."
4,4,16578,Jason West,Licencia,631-57-6208,1654 Adams Spring Ap,81139864,matthewcarr@yahoo.co,"[b'Z', b'3', b'f', b'D', b'8', b'o', b'2', b'b..."
...,...,...,...,...,...,...,...,...,...
9995,9995,16578,Ann Martin,DPI,590-34-9478,55399 Ryan Track\nDan,39241256,roachwendy@gmail.com,"[b'Z', b'v', b'I', b'R', b'g', b'T', b'1', b'X..."
9996,9996,16578,Rickey Cruz,Licencia,683-13-5659,799 Barnes Rue\nNorth,57820290,susan55@hotmail.com,"[b'B', b'H', b'N', b'O', b'S', b'9', b'N', b'm..."
9997,9997,14586,Hannah Sanchez,Licencia,681-85-8890,059 Patty Island\nWes,11249491,rmorgan@yahoo.com,"[b'!', b'e', b'6', b'7', b'R', b'd', b'h', b'B..."
9998,9998,18568,Brian Orr,Licencia,846-47-3146,95472 Grimes Street\n,68817726,patriciabrown@yahoo.,"[b'J', b'O', b'%', b'x', b'(', b'D', b'b', b'd..."


#### 2.3.2 Tabla rol

In [13]:
query = 'SELECT * FROM rol;'
df_rol = pd.read_sql(query, driver)
df_rol

,id_rol,nombre,descripcion,estado_rol
0,14586,Gerente,Acceso maximo,True
1,18568,Administrador,Acceso medio,True
2,16578,Cliente,Acceso minimo,True


In [14]:
df_rol = df_rol.drop(columns=['estado_rol'])
df_rol = df_rol.rename(columns={'nombre': 'nombre_rol', 'descripcion': 'descripcion_rol'})
df_rol

,id_rol,nombre_rol,descripcion_rol
0,14586,Gerente,Acceso maximo
1,18568,Administrador,Acceso medio
2,16578,Cliente,Acceso minimo


#### 2.3.3 Crear la dimension

In [15]:
dim_usuario = df_usuario.merge(df_rol, on='id_rol', how='inner')
dim_usuario

,id_usuario,id_rol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,nombre_rol,descripcion_rol
0,0,14586,Jason Collins,Licencia,171-37-6240,167 Kim Mills Suite,66466083,josephaustin@gmail.c,"[b'x', b'6', b'3', b'n', b'F', b'V', b'T', b'y...",Gerente,Acceso maximo
1,1,14586,Kelly Nelson,DPI,317-88-6562,55279 Gibson Forge S,22196109,cheryl25@hotmail.com,"[b'&', b'A', b'J', b'd', b'9', b'O', b'k', b'L...",Gerente,Acceso maximo
2,3,14586,Christopher Hardy,DPI,084-04-7482,05077 Thomas Square\n,27287390,amanda48@gmail.com,"[b')', b'm', b'6', b'5', b'7', b'v', b'a', b'p...",Gerente,Acceso maximo
3,6,14586,Karen Martinez,Licencia,468-80-7914,473 Jeffrey Road Sui,50765566,kyle37@gmail.com,"[b'6', b'B', b'$', b'r', b'V', b'1', b'G', b'h...",Gerente,Acceso maximo
4,7,14586,Marilyn Smith,DPI,393-69-7839,8914 Matthew Straven,47257080,hannahherrera@hotmai,"[b'u', b'*', b')', b'1', b'm', b'$', b'I', b'e...",Gerente,Acceso maximo
...,...,...,...,...,...,...,...,...,...,...,...
9995,9984,16578,Angela Mitchell,Licencia,376-43-4261,6838 Simon Expresswa,95455456,james52@yahoo.com,"[b')', b'I', b'2', b'w', b'%', b'X', b'Y', b'o...",Cliente,Acceso minimo
9996,9987,16578,Kathryn Lopez,Licencia,177-53-2078,9983 Hammond Mill\nSo,99755621,williammurray@hotmai,"[b'(', b'W', b'f', b'j', b'j', b'f', b'C', b'+...",Cliente,Acceso minimo
9997,9989,16578,Janet Kelley,Licencia,440-43-8139,8241 Maddox Spurs\nBr,52289217,hmyers@yahoo.com,"[b'I', b'8', b'I', b'r', b'Y', b'N', b'C', b'Z...",Cliente,Acceso minimo
9998,9995,16578,Ann Martin,DPI,590-34-9478,55399 Ryan Track\nDan,39241256,roachwendy@gmail.com,"[b'Z', b'v', b'I', b'R', b'g', b'T', b'1', b'X...",Cliente,Acceso minimo


### 2.4 Dimensión de fecha

#### 2.4.1 Traer fechas de la tabla venta

In [16]:
query = 'SELECT fecha FROM venta;'
df_fecha_venta = pd.read_sql(query, driver)
df_fecha_venta

,fecha
0,2022-01-02
1,2024-02-06
2,2024-03-26
3,2022-01-29
4,2021-06-08
...,...
995,2023-04-10
996,2021-12-19
997,2022-02-09
998,2020-05-22


#### 2.4.2 Traer fechas de la tabla ingreso

In [17]:
query = 'SELECT fecha FROM ingreso;'
df_fecha_ingreso = pd.read_sql(query, driver)
df_fecha_ingreso

,fecha
0,2020-02-13
1,2024-02-09
2,2022-05-17
3,2022-10-27
4,2022-12-21
...,...
995,2023-12-16
996,2022-11-22
997,2022-08-04
998,2022-04-07


#### 2.4.3 Unificar en un solo dataframe

In [18]:
df_date = pd.concat([df_fecha_venta, df_fecha_ingreso])
df_date

,fecha
0,2022-01-02
1,2024-02-06
2,2024-03-26
3,2022-01-29
4,2021-06-08
...,...
995,2023-12-16
996,2022-11-22
997,2022-08-04
998,2022-04-07


#### 2.4.4 Eliminar registros duplicados

In [19]:
df_date = df_date.drop_duplicates()
df_date

,fecha
0,2022-01-02
1,2024-02-06
2,2024-03-26
3,2022-01-29
4,2021-06-08
...,...
971,2020-01-22
976,2022-03-26
979,2020-01-02
980,2024-03-11


#### 2.4.5 Crear los fields para la dimensión

In [20]:
df_date = df_date.copy()
df_date['año'] = pd.DatetimeIndex(df_date['fecha']).year
df_date['trimestre'] = pd.DatetimeIndex(df_date['fecha']).quarter
df_date['mes'] = pd.DatetimeIndex(df_date['fecha']).month
df_date['dia'] = pd.DatetimeIndex(df_date['fecha']).day
df_date['dia_de_la_semana'] = pd.DatetimeIndex(df_date['fecha']).dayofweek
df_date['es_fin_de_semana'] = df_date['fecha'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)
df_date['id_fecha'] = df_date['fecha'].astype(str).str.replace('-', '')
df_date

,fecha,año,trimestre,mes,dia,dia_de_la_semana,es_fin_de_semana,id_fecha
0,2022-01-02,2022,1,1,2,6,1,20220102
1,2024-02-06,2024,1,2,6,1,0,20240206
2,2024-03-26,2024,1,3,26,1,0,20240326
3,2022-01-29,2022,1,1,29,5,1,20220129
4,2021-06-08,2021,2,6,8,1,0,20210608
...,...,...,...,...,...,...,...,...
971,2020-01-22,2020,1,1,22,2,0,20200122
976,2022-03-26,2022,1,3,26,5,1,20220326
979,2020-01-02,2020,1,1,2,3,0,20200102
980,2024-03-11,2024,1,3,11,0,0,20240311


### 2.5 Tabla de Hechos - Ventas

In [21]:
query = 'SELECT * FROM detalle_venta;'
df_hecho_ventas_det = pd.read_sql(query, driver)
df_hecho_ventas_det

,id_detalle_venta,id_venta,id_articulo,cantidad,precio,descuento
0,0,995,273,52,47.75,0.29
1,1,483,685,329,55.95,0.78
2,2,110,372,815,72.39,0.11
3,3,312,571,293,87.07,0.47
4,4,57,773,709,95.64,0.86
...,...,...,...,...,...,...
2995,2995,604,296,405,52.07,0.49
2996,2996,311,592,194,14.28,0.98
2997,2997,672,434,8,14.50,0.61
2998,2998,378,903,689,54.63,0.71


In [22]:
query = 'SELECT id_venta, id_persona, id_usuario, fecha FROM venta;'
df_hecho_ventas = pd.read_sql(query, driver)
df_hecho_ventas

,id_venta,id_persona,id_usuario,fecha
0,0,5,1395,2022-01-02
1,1,8,9572,2024-02-06
2,2,5,1973,2024-03-26
3,3,8,5543,2022-01-29
4,4,1,4457,2021-06-08
...,...,...,...,...
995,995,8,563,2023-04-10
996,996,6,2250,2021-12-19
997,997,5,7239,2022-02-09
998,998,3,4264,2020-05-22


In [23]:
fact_ventas = df_hecho_ventas_det.merge(df_hecho_ventas, on='id_venta', how='inner')
fact_ventas

,id_detalle_venta,id_venta,id_articulo,cantidad,precio,descuento,id_persona,id_usuario,fecha
0,0,995,273,52,47.75,0.29,8,563,2023-04-10
1,159,995,948,149,31.18,0.10,8,563,2023-04-10
2,2342,995,178,240,77.36,0.13,8,563,2023-04-10
3,1,483,685,329,55.95,0.78,3,5686,2023-05-10
4,1188,483,987,803,38.36,0.79,3,5686,2023-05-10
...,...,...,...,...,...,...,...,...,...
2995,2887,73,314,498,47.76,0.89,5,852,2021-09-04
2996,2906,287,950,382,96.01,0.60,1,7167,2020-12-13
2997,2941,502,623,564,76.70,0.47,4,8083,2021-09-01
2998,2955,554,167,13,57.66,0.85,3,7850,2022-06-04


#### 2.5.2 Agregar columnas computadas (impuesto, total y fecha)

In [24]:
fact_ventas['total'] = fact_ventas['cantidad'] * (fact_ventas['precio'] - fact_ventas['descuento'])
fact_ventas

,id_detalle_venta,id_venta,id_articulo,cantidad,precio,descuento,id_persona,id_usuario,fecha,total
0,0,995,273,52,47.75,0.29,8,563,2023-04-10,2467.92
1,159,995,948,149,31.18,0.10,8,563,2023-04-10,4630.92
2,2342,995,178,240,77.36,0.13,8,563,2023-04-10,18535.20
3,1,483,685,329,55.95,0.78,3,5686,2023-05-10,18150.93
4,1188,483,987,803,38.36,0.79,3,5686,2023-05-10,30168.71
...,...,...,...,...,...,...,...,...,...,...
2995,2887,73,314,498,47.76,0.89,5,852,2021-09-04,23341.26
2996,2906,287,950,382,96.01,0.60,1,7167,2020-12-13,36446.62
2997,2941,502,623,564,76.70,0.47,4,8083,2021-09-01,42993.72
2998,2955,554,167,13,57.66,0.85,3,7850,2022-06-04,738.53


In [25]:
fact_ventas['impuesto'] = fact_ventas['total'] * 0.12
fact_ventas

,id_detalle_venta,id_venta,id_articulo,cantidad,precio,descuento,id_persona,id_usuario,fecha,total,impuesto
0,0,995,273,52,47.75,0.29,8,563,2023-04-10,2467.92,296.1504
1,159,995,948,149,31.18,0.10,8,563,2023-04-10,4630.92,555.7104
2,2342,995,178,240,77.36,0.13,8,563,2023-04-10,18535.20,2224.2240
3,1,483,685,329,55.95,0.78,3,5686,2023-05-10,18150.93,2178.1116
4,1188,483,987,803,38.36,0.79,3,5686,2023-05-10,30168.71,3620.2452
...,...,...,...,...,...,...,...,...,...,...,...
2995,2887,73,314,498,47.76,0.89,5,852,2021-09-04,23341.26,2800.9512
2996,2906,287,950,382,96.01,0.60,1,7167,2020-12-13,36446.62,4373.5944
2997,2941,502,623,564,76.70,0.47,4,8083,2021-09-01,42993.72,5159.2464
2998,2955,554,167,13,57.66,0.85,3,7850,2022-06-04,738.53,88.6236


In [26]:
df_factTable = fact_ventas.copy()
df_factTable['year'] = pd.DatetimeIndex(df_factTable['fecha']).year
df_factTable['month'] = pd.DatetimeIndex(df_factTable['fecha']).month
df_factTable['day'] = pd.DatetimeIndex(df_factTable['fecha']).day
df_factTable['hour'] = pd.DatetimeIndex(df_factTable['fecha']).hour
df_factTable['minute'] = pd.DatetimeIndex(df_factTable['fecha']).minute
fact_ventas['id_fecha'] = fact_ventas['fecha'].astype(str).str.replace('-', '')


fact_ventas.head()

,id_detalle_venta,id_venta,id_articulo,cantidad,precio,descuento,id_persona,id_usuario,fecha,total,impuesto,id_fecha
0,0,995,273,52,47.75,0.29,8,563,2023-04-10,2467.92,296.1504,20230410
1,159,995,948,149,31.18,0.10,8,563,2023-04-10,4630.92,555.7104,20230410
2,2342,995,178,240,77.36,0.13,8,563,2023-04-10,18535.20,2224.2240,20230410
3,1,483,685,329,55.95,0.78,3,5686,2023-05-10,18150.93,2178.1116,20230510
4,1188,483,987,803,38.36,0.79,3,5686,2023-05-10,30168.71,3620.2452,20230510


In [ ]:
fact_ventas = fact_ventas.drop(columns=['estado_rol'])
fact_ventas.head()

In [27]:
fact_ventas['id_articulo'].value_counts()

id_articulo
531    9
925    9
716    9
174    9
130    9
      ..
170    1
591    1
359    1
432    1
949    1
Name: count, Length: 938, dtype: int64

### 2.6 Tabla de Hechos - Ingresos

In [28]:
query = 'SELECT * FROM detalle_ingreso;'
df_hecho_ingreso_det = pd.read_sql(query, driver)
df_hecho_ingreso_det

,id_detalle_ingreso,id_ingreso,id_articulo,cantidad,precio_unitario
0,0,817,155,113,24.47
1,1,772,534,425,38.95
2,2,56,250,389,75.60
3,3,416,531,595,23.93
4,4,532,952,767,93.92
...,...,...,...,...,...
2995,2995,200,984,526,54.57
2996,2996,686,35,218,81.29
2997,2997,818,253,602,21.87
2998,2998,432,491,249,71.64


In [29]:
query = 'SELECT id_ingreso, id_proveedor, id_usuario, fecha FROM ingreso;'
df_hecho_ingreso = pd.read_sql(query, driver)
df_hecho_ingreso

,id_ingreso,id_proveedor,id_usuario,fecha
0,0,3,4808,2020-02-13
1,1,7,8855,2024-02-09
2,2,7,6980,2022-05-17
3,3,2,9179,2022-10-27
4,4,4,8155,2022-12-21
...,...,...,...,...
995,995,7,1624,2023-12-16
996,996,7,1965,2022-11-22
997,997,9,4704,2022-08-04
998,998,2,4117,2022-04-07


In [30]:
fact_ingresos = df_hecho_ingreso_det.merge(df_hecho_ingreso, on='id_ingreso', how='inner')
fact_ingresos

,id_detalle_ingreso,id_ingreso,id_articulo,cantidad,precio_unitario,id_proveedor,id_usuario,fecha
0,0,817,155,113,24.47,7,9916,2022-12-28
1,91,817,15,674,71.85,7,9916,2022-12-28
2,1587,817,419,689,41.60,7,9916,2022-12-28
3,2935,817,717,194,33.75,7,9916,2022-12-28
4,1,772,534,425,38.95,9,8011,2022-04-10
...,...,...,...,...,...,...,...,...
2995,2926,113,107,542,34.85,1,4296,2023-09-29
2996,2947,444,76,765,74.01,8,5809,2020-08-15
2997,2949,811,680,4,91.33,2,9962,2020-03-29
2998,2955,476,652,584,43.24,6,6772,2022-12-25


#### 2.6.2 Agregar columnas computadas (total y id_fecha)

In [31]:
fact_ingresos['total'] = fact_ingresos['cantidad'] * fact_ingresos['precio_unitario']
fact_ingresos

,id_detalle_ingreso,id_ingreso,id_articulo,cantidad,precio_unitario,id_proveedor,id_usuario,fecha,total
0,0,817,155,113,24.47,7,9916,2022-12-28,2765.11
1,91,817,15,674,71.85,7,9916,2022-12-28,48426.90
2,1587,817,419,689,41.60,7,9916,2022-12-28,28662.40
3,2935,817,717,194,33.75,7,9916,2022-12-28,6547.50
4,1,772,534,425,38.95,9,8011,2022-04-10,16553.75
...,...,...,...,...,...,...,...,...,...
2995,2926,113,107,542,34.85,1,4296,2023-09-29,18888.70
2996,2947,444,76,765,74.01,8,5809,2020-08-15,56617.65
2997,2949,811,680,4,91.33,2,9962,2020-03-29,365.32
2998,2955,476,652,584,43.24,6,6772,2022-12-25,25252.16


In [32]:
df_factTable2 = fact_ingresos.copy()
df_factTable2['year'] = pd.DatetimeIndex(df_factTable2['fecha']).year
df_factTable2['month'] = pd.DatetimeIndex(df_factTable2['fecha']).month
df_factTable2['day'] = pd.DatetimeIndex(df_factTable2['fecha']).day
df_factTable2['hour'] = pd.DatetimeIndex(df_factTable2['fecha']).hour
df_factTable2['minute'] = pd.DatetimeIndex(df_factTable2['fecha']).minute
fact_ingresos['id_fecha'] = fact_ingresos['fecha'].astype(str).str.replace('-', '')


fact_ingresos.head()

,id_detalle_ingreso,id_ingreso,id_articulo,cantidad,precio_unitario,id_proveedor,id_usuario,fecha,total,id_fecha
0,0,817,155,113,24.47,7,9916,2022-12-28,2765.11,20221228
1,91,817,15,674,71.85,7,9916,2022-12-28,48426.90,20221228
2,1587,817,419,689,41.60,7,9916,2022-12-28,28662.40,20221228
3,2935,817,717,194,33.75,7,9916,2022-12-28,6547.50,20221228
4,1,772,534,425,38.95,9,8011,2022-04-10,16553.75,20220410


### 3.1 Hacemos configuración inicial

In [33]:
config = configparser.ConfigParser()
config.read('config2.cfg')

['config2.cfg']

In [34]:
aws_rds_conn = boto3.client('rds', aws_access_key_id = config.get('IAM', 'ACCESS_KEY'), 
                              aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY'),
                              region_name = 'us-east-1')

In [35]:
rds_instances_ids = []
aws_response = aws_rds_conn.describe_db_instances()

for response in aws_response['DBInstances']:
    rds_instances_ids.append(response['DBInstanceIdentifier'])

print(f'Instancias Disponibles: {rds_instances_ids}')

Instancias Disponibles: ['e-dreams-dwh']


### 3.2 Creamos instancia de base de datos en AWS - mySQL

In [36]:
try:
    response = aws_rds_conn.create_db_instance(
            DBInstanceIdentifier = config.get('DWH', 'DWH_INSTANCE_ID'),
            DBName = config.get('DWH', 'DWH_NAME'),
            DBInstanceClass = 'db.t3.micro',
            Engine = 'mysql',
            MasterUsername = config.get('DWH', 'DWH_USER'),
            MasterUserPassword = config.get('DWH', 'DWH_PASSWORD'),
            Port = int(config.get('DWH', 'DWH_PORT')),
            PubliclyAccessible = True,
            VpcSecurityGroupIds = [config.get('VPC', 'SECURITY_GROUP')],
            AllocatedStorage = 15
        )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print('La instancia ya existe')
except Exception as ex:
    print('Error!', ex)

La instancia ya existe


In [37]:
try:

    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier = config.get('DWH', 'DWH_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)

except Exception as ex:
    print('Error!!!', ex)


e-dreams-dwh.ch62gk4wowqh.us-east-1.rds.amazonaws.com


In [38]:
# Delete the DB instance
#response = aws_rds_conn.delete_db_instance(
#    DBInstanceIdentifier=config.get('DWH', 'DWH_INSTANCE_ID'),
#    SkipFinalSnapshot=True  # Set to False if you want to take a final snapshot
#)

In [39]:
try:

    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier = config.get('DWH', 'DWH_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)

except Exception as ex:
    print('Error!!!', ex)

e-dreams-dwh.ch62gk4wowqh.us-east-1.rds.amazonaws.com


### 3.3 Creamos la bd para el DWH

In [45]:
# pip install mysql-connector-python
# pip install pymysql
import mysql.connector

In [46]:
import sql_queries_2
sql_queries_2.DDL_QUERY

'\nCREATE TABLE IF NOT EXISTS dim_articulo (\n    id_articulo INT,\n\tid_categoria INT,\n    codigo VARCHAR(50),\n    nombre VARCHAR(100),\n    precio_venta NUMERIC(11,2),\n    descripcion VARCHAR(255),\n    imagen VARCHAR(20),\n    nombre_categoria VARCHAR(50),\n    descripcion_categoria VARCHAR(255),\n    \n    PRIMARY KEY (id_articulo)\n);\n\nCREATE TABLE IF NOT EXISTS dim_persona (\n    id_persona INT,\n\ttipo_persona VARCHAR(20),\n    nombre VARCHAR(100),\n    tipo_documento VARCHAR(20),\n    num_documento VARCHAR(20),\n    direccion VARCHAR(20),\n    telefono VARCHAR(20),\n    email VARCHAR(20),\n    \n    PRIMARY KEY (id_persona)\n);\n\nCREATE TABLE IF NOT EXISTS dim_usuario (\n    id_usuario INT,\n\tid_rol INT,\n    nombre VARCHAR(100),\n    tipo_documento VARCHAR(20),\n    num_documento VARCHAR(20),\n    direccion VARCHAR(20),\n    telefono VARCHAR(20),\n    email VARCHAR(20),\n    clave BLOB ,\n    nombre_rol VARCHAR(30),\n    descripcion_rol VARCHAR(30),\n    \n    PRIMARY K

In [47]:
try:
    db_sql_conn = mysql.connector.connect(
        database=config.get('DWH', 'DWH_NAME'),
        user=config.get('DWH', 'DWH_USER'),
        password=config.get('DWH', 'DWH_PASSWORD'),
        host=RDS_HOSTNAME,
        port=int(config.get('DWH', 'DWH_PORT'))
    )
    cursor = db_sql_conn.cursor()
    cursor.execute(sql_queries_2.DDL_QUERY, multi=True)
    db_sql_conn.commit()
    cursor.close()
    print('Base de Datos Creada Exitosamente')

except Exception as ex:
    print('Error!!!', ex)

Error!!! Commands out of sync; you can't run this command now


In [48]:
mysql_driver = f"""mysql+pymysql://{config.get('DWH', 'DWH_USER')}:{config.get('DWH', 'DWH_PASSWORD')}@{RDS_HOSTNAME}:{config.get('DWH', 'DWH_PORT')}/{config.get('DWH', 'DWH_NAME')}"""  

### 3.4 Poblar las dimensiones

In [49]:
dim_articulo.to_sql('dim_articulo', mysql_driver, index=False, if_exists='append')

1000

In [50]:
dim_personas.to_sql('dim_persona', mysql_driver, index=False, if_exists='append')

10

In [51]:
dim_usuario.to_sql('dim_usuario', mysql_driver, index=False, if_exists='append')

10000

In [52]:
df_date.to_sql('dim_fecha', mysql_driver, index=False, if_exists='append')

1120

In [53]:
fact_ventas.to_sql('fact_ventas', mysql_driver, index=False, if_exists='append')

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'fecha' in 'field list'")
[SQL: INSERT INTO fact_ventas (id_detalle_venta, id_venta, id_articulo, cantidad, precio, descuento, id_persona, id_usuario, fecha, total, impuesto, id_fecha) VALUES (%(id_detalle_venta)s, %(id_venta)s, %(id_articulo)s, %(cantidad)s, %(precio)s, %(descuento)s, %(id_persona)s, %(id_usuario)s, %(fecha)s, %(total)s, %(impuesto)s, %(id_fecha)s)]
[parameters: ({'id_detalle_venta': 0, 'id_venta': 995, 'id_articulo': 273, 'cantidad': 52, 'precio': 47.75, 'descuento': 0.29, 'id_persona': 8, 'id_usuario': 563, 'fecha': datetime.datetime(2023, 4, 10, 0, 0), 'total': 2467.92, 'impuesto': 296.1504, 'id_fecha': '20230410'}, {'id_detalle_venta': 159, 'id_venta': 995, 'id_articulo': 948, 'cantidad': 149, 'precio': 31.18, 'descuento': 0.1, 'id_persona': 8, 'id_usuario': 563, 'fecha': datetime.datetime(2023, 4, 10, 0, 0), 'total': 4630.92, 'impuesto': 555.7103999999999, 'id_fecha': '20230410'}, {'id_detall